## Задание 1

**Написать теггер на данных с руским языком**
1. проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации  
2. написать свой теггер как на занятии, но улучшить попробовать разные векторайзеры, добавить знание не только букв и слов но и совместно объединить эти признаки  
3. вместо векторайзеров взять эмбединги попробовать (word2vec и fasttext по желанию дополнительно можно взять tf.keras.layers.Embedding)  
4. взять не только эмбединги каждого слова, но и взять соседей, т.е. информацию о соседях количество соседей выбрать самим (узнать наилучшее количество соседей)    
5. сравнить все реализованные методы сделать выводы

# 1.

## загрузка данных

In [1]:
import pyconll
import nltk
from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger, TrigramTagger
from nltk.tag import RegexpTagger

from gensim.models import Word2Vec, FastText
#import gensim.downloader as api
#word_vectors = api.load("fasttext-wiki-news-subwords-300")

c:\nlp_test\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
full_train = pyconll.load_from_file('datasets/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('datasets/ru_syntagrus-ud-dev.conllu')

In [4]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Анкета NOUN
. PUNCT

Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT



In [5]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])
    
    
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 205
Наибольшая длина токена 47


In [6]:
for sent in full_train:
    for token in sent:
        if len(token.form) == 47:
            print((token.form, token.upos))
            break


('sendsms.megafon.ru/status/send/0123456789ABCDEF', 'X')


In [7]:
default_tagger = nltk.DefaultTagger('NOUN')
default_tagger.evaluate(fdata_test)

0.23568564014423887

In [8]:
unigram_tagger = UnigramTagger(fdata_train)
unigram_tagger.evaluate(fdata_test)

0.8772537323492737

In [9]:
bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
bigram_tagger.evaluate(fdata_test)

0.8829828463586425

In [10]:
trigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger)
trigram_tagger.evaluate(fdata_test)

0.882081353418933

In [11]:
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NOUN') 
tag = backoff_tagger(fdata_train,  
                     [
                      UnigramTagger, 
                      BigramTagger, 
                      TrigramTagger
                     ],  
                     backoff = backoff) 
  
tag.evaluate(fdata_test) 

0.9119991237825633

# 2

In [12]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [13]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])
        
        
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label) 
test_enc_labels = le.transform(test_label)
le.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN',
       'NO_TAG', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM',
       'VERB', 'X'], dtype='<U6')

In [14]:
for vectorizer in [CountVectorizer, HashingVectorizer, TfidfVectorizer]:

    scaler = StandardScaler(with_mean=False)
    coder = vectorizer(ngram_range=(1, 5), analyzer='char')
    

    X_train = coder.fit_transform(train_tok)
    X_test = coder.transform(test_tok)
    
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)    
    
    
    print(X_train.shape)
    lr = LogisticRegression(random_state=0, max_iter = 100, n_jobs=7)
    lr.fit(X_train, train_enc_labels)

    pred = lr.predict(X_test)

    print(vectorizer, accuracy_score(test_enc_labels, pred))

(871526, 149809)
<class 'sklearn.feature_extraction.text.CountVectorizer'> 0.9435261011694133
(871526, 1048576)
<class 'sklearn.feature_extraction.text.HashingVectorizer'> 0.9471657735988946
(871526, 149809)
<class 'sklearn.feature_extraction.text.TfidfVectorizer'> 0.9487497051191319


In [15]:
for vectorizer in [CountVectorizer, HashingVectorizer, TfidfVectorizer]:

    scaler = StandardScaler(with_mean=False)
    coder = vectorizer(ngram_range=(1, 5), analyzer='word')
    

    X_train = coder.fit_transform(train_tok)
    X_test = coder.transform(test_tok)
    
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)    
    
    
    print(X_train.shape)
    lr = LogisticRegression(random_state=0, max_iter = 100, n_jobs=7)
    lr.fit(X_train, train_enc_labels)

    pred = lr.predict(X_test)

    print(vectorizer, accuracy_score(test_enc_labels, pred))

(871526, 99485)
<class 'sklearn.feature_extraction.text.CountVectorizer'> 0.7532184140464395
(871526, 1048576)
<class 'sklearn.feature_extraction.text.HashingVectorizer'> 0.7722255922892866
(871526, 99485)
<class 'sklearn.feature_extraction.text.TfidfVectorizer'> 0.7532605398847437


In [16]:
import numpy as np
from scipy.sparse import hstack
scaler = StandardScaler(with_mean=False)
coder_1 = TfidfVectorizer(ngram_range=(1, 5), analyzer='char')
coder_2 = HashingVectorizer(ngram_range=(1, 5), analyzer='word')

X_train_1 = coder_1.fit_transform(train_tok)
X_test_1 = coder_1.transform(test_tok)

X_train_2 = coder_2.fit_transform(train_tok)
X_test_2 = coder_2.transform(test_tok)


X_train = hstack((X_train_1,X_train_2))
X_test = hstack((X_test_1,X_test_2))

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)    


print(X_train.shape)
lr = LogisticRegression(random_state=0, max_iter = 100, n_jobs=7)
lr.fit(X_train, train_enc_labels)

pred = lr.predict(X_test)

print('TfidfVectorizer_char + HashingVectorizer_word :', accuracy_score(test_enc_labels, pred))

(871526, 1198385)
TfidfVectorizer_char + HashingVectorizer_word : 0.9441327132409935


# 3

In [17]:
from pymorphy2 import MorphAnalyzer
from tqdm import tqdm_notebook


morpher = MorphAnalyzer()

just_texts_trian=[]
just_poss_train =[]
for sent in tqdm_notebook(fdata_train):
    just_text_sent = []
    just_pos_sent = []
    for point in sent:
        just_text_sent.append(morpher.parse(point[0].lower())[0].normal_form)
        just_pos_sent.append(point[1])
    
    just_texts_trian.append(just_text_sent)
    just_poss_train.append(just_pos_sent)
        
    
print(just_texts_trian[-1])
print(just_poss_train[-1])


just_texts_test=[]
just_poss_test =[]
for sent in tqdm_notebook(fdata_test):
    just_text_sent = []
    just_pos_sent = []
    for point in sent:
        just_text_sent.append(morpher.parse(point[0].lower())[0].normal_form)        
        just_pos_sent.append(point[1])
    
    just_texts_test.append(just_text_sent)
    just_poss_test.append(just_pos_sent)
        
    
print(just_texts_test[-1])
print(just_poss_test[-1])

  0%|          | 0/48814 [00:00<?, ?it/s]

['но', 'кто-то', 'идти', 'я', 'навстречу', 'и', ',', 'мочь', 'быть', ',', 'пройти', 'уже', 'пол', 'путь', '…']
['CCONJ', 'PRON', 'VERB', 'PRON', 'ADV', 'CCONJ', 'PUNCT', 'VERB', 'VERB', 'PUNCT', 'VERB', 'ADV', 'NUM', 'NOUN', 'PUNCT']


  0%|          | 0/6584 [00:00<?, ?it/s]

['поэтому', 'ничего', 'не', 'измениться', '.']
['ADV', 'PRON', 'PART', 'VERB', 'PUNCT']


In [18]:
sentences = just_texts_trian

dimention = 200   # int(39694**(1/2)) = 200

modelW2V = Word2Vec(sentences=sentences, vector_size=dimention, window=5, min_count=1)
modelFT = FastText(sentences=sentences, window=5, min_count=1, vector_size=dimention)

In [19]:
def get_vectorized_data(model, num=1):
    train_tok = []
    train_words = []
    train_label = []
    
    for i in range(len(just_texts_trian)):
        for j in range(len(just_texts_trian[i])):
            nums_list=[]
            truncated_sent =''
            try:
            
                for k in range(num):
                    if (j >= k):
                        result = model.wv[just_texts_trian[i][j-k]]
                        word_result = just_texts_trian[i][j-k]
                    else:
                        result = np.zeros(dimention)
                        word_result = '-'
                        
                    nums_list = nums_list + list(result)
                    truncated_sent = truncated_sent + word_result + ' '
                    
                train_tok.append(np.array(nums_list))
                train_words.append(truncated_sent)    
                train_label.append(just_poss_train[i][j])
            except:
                    pass


    test_tok = []
    test_words = []    
    test_label = []            
    for i in range(len(just_texts_test)):
        for j in range(len(just_texts_test[i])):
            nums_list=[]
            truncated_sent =''
            try:
            
                for k in range(num):
                    if (j >= k):
                        result = model.wv[just_texts_test[i][j-k]]
                        word_result = just_texts_test[i][j-k]
                    else:
                        result = np.zeros(dimention)
                        word_result = '-'
                        
                    nums_list = nums_list + list(result)
                    truncated_sent = truncated_sent + word_result + ' '
                    
                test_tok.append(np.array(nums_list))
                test_words.append(truncated_sent)                        
                test_label.append(just_poss_test[i][j])
            except:
                    pass

    return train_tok, train_label, test_tok, test_label, train_words, test_words

In [20]:
#train_tok, train_label, test_tok, test_label, train_words, test_words = get_vectorized_data(model, num=3)

In [21]:
#print(len(train_label), len(train_tok), len(test_label), len(test_tok))

In [22]:
#test_words

In [23]:
def run_test(model, num=1):
    train_tok, train_label, test_tok, test_label, train_words, test_words = get_vectorized_data(model, num=num)
    print(len(train_label), len(train_tok), len(test_label), len(test_tok))

    le = LabelEncoder()
    train_enc_labels = le.fit_transform(train_label) 
    test_enc_labels = le.transform(test_label)
    print(le.classes_)


    scaler = StandardScaler()
    train_tok = scaler.fit_transform(train_tok)
    test_tok = scaler.fit_transform(test_tok)    


    lr = LogisticRegression(random_state=0, max_iter = 100, verbose=1, n_jobs=-1)
    lr.fit(train_tok, train_enc_labels)

    pred = lr.predict(test_tok)

    print('LogisticRegression', 'word vector bild on', num, 'item(s)')
    print(model)
    print('Scor: ', accuracy_score(test_enc_labels, pred))    

# Протестируем FastText с разными эмбедингами.

In [24]:
run_test(modelFT, num=1)

871526 871526 118692 118692
['ADJ' 'ADP' 'ADV' 'AUX' 'CCONJ' 'DET' 'INTJ' 'NOUN' 'NUM' 'PART' 'PRON'
 'PROPN' 'PUNCT' 'SCONJ' 'SYM' 'VERB' 'X' None]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  3.2min finished


LogisticRegression word vector bild on 1 item(s)
FastText(vocab=39694, vector_size=200, alpha=0.025)
Scor:  0.901914198092542


In [25]:
run_test(modelFT, num=2)

871526 871526 118692 118692
['ADJ' 'ADP' 'ADV' 'AUX' 'CCONJ' 'DET' 'INTJ' 'NOUN' 'NUM' 'PART' 'PRON'
 'PROPN' 'PUNCT' 'SCONJ' 'SYM' 'VERB' 'X' None]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  5.2min finished


LogisticRegression word vector bild on 2 item(s)
FastText(vocab=39694, vector_size=200, alpha=0.025)
Scor:  0.9022512047989755


In [26]:
run_test(modelFT, num=3)

871526 871526 118692 118692
['ADJ' 'ADP' 'ADV' 'AUX' 'CCONJ' 'DET' 'INTJ' 'NOUN' 'NUM' 'PART' 'PRON'
 'PROPN' 'PUNCT' 'SCONJ' 'SYM' 'VERB' 'X' None]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  7.6min finished


LogisticRegression word vector bild on 3 item(s)
FastText(vocab=39694, vector_size=200, alpha=0.025)
Scor:  0.9012233343443534


In [27]:
run_test(modelFT, num=4)

871526 871526 118692 118692
['ADJ' 'ADP' 'ADV' 'AUX' 'CCONJ' 'DET' 'INTJ' 'NOUN' 'NUM' 'PART' 'PRON'
 'PROPN' 'PUNCT' 'SCONJ' 'SYM' 'VERB' 'X' None]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed: 10.6min finished


LogisticRegression word vector bild on 4 item(s)
FastText(vocab=39694, vector_size=200, alpha=0.025)
Scor:  0.9018046709129511


In [28]:
run_test(modelFT, num=5)

871526 871526 118692 118692
['ADJ' 'ADP' 'ADV' 'AUX' 'CCONJ' 'DET' 'INTJ' 'NOUN' 'NUM' 'PART' 'PRON'
 'PROPN' 'PUNCT' 'SCONJ' 'SYM' 'VERB' 'X' None]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed: 13.3min finished


LogisticRegression word vector bild on 5 item(s)
FastText(vocab=39694, vector_size=200, alpha=0.025)
Scor:  0.9018720722542378


# Протестируем Word2Vec с разными эмбедингами.

In [29]:
run_test(modelW2V, num=1)

871526 871526 114586 114586
['ADJ' 'ADP' 'ADV' 'AUX' 'CCONJ' 'DET' 'INTJ' 'NOUN' 'NUM' 'PART' 'PRON'
 'PROPN' 'PUNCT' 'SCONJ' 'SYM' 'VERB' 'X' None]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  3.1min finished


LogisticRegression word vector bild on 1 item(s)
Word2Vec(vocab=39694, vector_size=200, alpha=0.025)
Scor:  0.8177875133087812


In [30]:
run_test(modelW2V, num=2)

871526 871526 110739 110739
['ADJ' 'ADP' 'ADV' 'AUX' 'CCONJ' 'DET' 'INTJ' 'NOUN' 'NUM' 'PART' 'PRON'
 'PROPN' 'PUNCT' 'SCONJ' 'SYM' 'VERB' 'X' None]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  5.3min finished


LogisticRegression word vector bild on 2 item(s)
Word2Vec(vocab=39694, vector_size=200, alpha=0.025)
Scor:  0.8294096930620648


In [31]:
run_test(modelW2V, num=3)

871526 871526 107686 107686
['ADJ' 'ADP' 'ADV' 'AUX' 'CCONJ' 'DET' 'INTJ' 'NOUN' 'NUM' 'PART' 'PRON'
 'PROPN' 'PUNCT' 'SCONJ' 'SYM' 'VERB' 'X' None]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  7.6min finished


LogisticRegression word vector bild on 3 item(s)
Word2Vec(vocab=39694, vector_size=200, alpha=0.025)
Scor:  0.8325594784837398


In [32]:
run_test(modelW2V, num=4)

871526 871526 105044 105044
['ADJ' 'ADP' 'ADV' 'AUX' 'CCONJ' 'DET' 'INTJ' 'NOUN' 'NUM' 'PART' 'PRON'
 'PROPN' 'PUNCT' 'SCONJ' 'SYM' 'VERB' 'X' None]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed: 10.7min finished


LogisticRegression word vector bild on 4 item(s)
Word2Vec(vocab=39694, vector_size=200, alpha=0.025)
Scor:  0.8317181371615704


In [33]:
run_test(modelW2V, num=5)

871526 871526 102719 102719
['ADJ' 'ADP' 'ADV' 'AUX' 'CCONJ' 'DET' 'INTJ' 'NOUN' 'NUM' 'PART' 'PRON'
 'PROPN' 'PUNCT' 'SCONJ' 'SYM' 'VERB' 'X' None]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed: 14.1min finished


LogisticRegression word vector bild on 5 item(s)
Word2Vec(vocab=39694, vector_size=200, alpha=0.025)
Scor:  0.8303235039281924


In [45]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Activation, Dense, Embedding, GlobalAveragePooling1D, Bidirectional, LSTM
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [60]:
vocab_size = 39694
sequence_length = 3

train_tok, train_label, test_tok, test_label, train_words, test_words = get_vectorized_data(modelFT, num=sequence_length)

le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label) 
test_enc_labels = le.transform(test_label)
le.classes_


array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM',
       'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X',
       None], dtype=object)

In [61]:
#test_words

In [63]:

# Use the text vectorization layer to normalize, split, and map strings to 
# Set maximum_sequence length as all samples are not of the same length.
vectorize_layer = TextVectorization(
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)



train_dataset = tf.data.Dataset.from_tensor_slices((train_words, train_enc_labels))
test_dataset  = tf.data.Dataset.from_tensor_slices((test_words, test_enc_labels))


BATCH_SIZE = 1024
BUFFER_SIZE = 10000
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
test_dataset = test_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)


print(train_dataset)
print(test_dataset)


text_ds = train_dataset.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

embedding_dim=200    # int(39694**(1/2)) = 200

model = Sequential([
  vectorize_layer,
  Embedding(vocab_size, embedding_dim, name="embedding"),
  Bidirectional(LSTM(units=32, return_sequences=True)),
  GlobalAveragePooling1D(),
  Dense(len(set(train_label)), activation=tf.keras.activations.softmax)
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])


model.fit(train_dataset,
    validation_data=test_dataset, 
    epochs=15)

<BatchDataset shapes: ((1024,), (1024,)), types: (tf.string, tf.int32)>
<BatchDataset shapes: ((1024,), (1024,)), types: (tf.string, tf.int32)>
Epoch 1/15
851/851 [==============================] - 88s 100ms/step - loss: 0.8171 - accuracy: 0.7279 - val_loss: 0.4792 - val_accuracy: 0.8103
Epoch 2/15
851/851 [==============================] - 85s 99ms/step - loss: 0.3528 - accuracy: 0.8521 - val_loss: 0.4485 - val_accuracy: 0.8213
Epoch 3/15
851/851 [==============================] - 86s 101ms/step - loss: 0.3111 - accuracy: 0.8672 - val_loss: 0.4512 - val_accuracy: 0.8231
Epoch 4/15
851/851 [==============================] - 85s 100ms/step - loss: 0.2908 - accuracy: 0.8752 - val_loss: 0.4564 - val_accuracy: 0.8243
Epoch 5/15
851/851 [==============================] - 85s 99ms/step - loss: 0.2750 - accuracy: 0.8815 - val_loss: 0.4622 - val_accuracy: 0.8253
Epoch 6/15
851/851 [==============================] - 85s 100ms/step - loss: 0.2620 - accuracy: 0.8866 - val_loss: 0.4709 - val_accur

# Результаты.

Для __nltk.tag__ лучший вариант это:
Комбинация из  __DefaultTagger UnigramTagger BigramTagger TrigramTagger__  
0.9119991237825633

Для __Vectorizer__ лучший вариант это:
Комбинация из __LogisticRegression__ поверх __TfidfVectorizer__ при условии __analyzer='char'__  
0.9487497051191319

Для __FastText__, __Word2Vec__ лучший вариант это:
Комбинация из __LogisticRegression__ поверх __FastText__ при условии эмбединга из __2-х слов__  
0.9022512047989755

Для __LSTM NN__ c __Embedding__ лучший вариант это:  
0.8260


# Вывод.

Для данной задачи побуквенный подход к кодированию слов выглядит предпочтительней чем по словный.